In [2]:
import numpy as np
import pandas as pd
import random
import cvxpy as cp
import math

In [9]:
# n : Number of agents
# m : Number of items

n = 3
m = 2

A = np.zeros(shape=(n, n*m))

beg = 0
end = m
for i in np.arange(n):
    idx = np.linspace(beg, end, m, endpoint = False)
    idx = idx.astype(int)
    A[i,idx] = 1
    beg += m
    end += m
    

print(A)

[[1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 1.]]


In [14]:
# x : The solution variable
# a : Parameter a
# A : Parameter A
# b : Parameter b, which is equal to the vector of size n including ones.

x = cp.Variable((n*m), integer = True)

a = cp.Parameter((m,n*n))
A_param = cp.Parameter((n, n*m))
b = cp.Parameter(n)

A_param.value = A
b.value = np.ones(shape=(n))
a = np.random.rand(m,n*n)
print(a)
print(A_param.shape)
print(x.shape)
print(b.shape)


[[0.91496063 0.65408427 0.44116778 0.96567247 0.15716513 0.47043394
  0.34680182 0.78950104 0.64472811]
 [0.03056815 0.95651386 0.49507638 0.16497192 0.91218557 0.35578395
  0.23264622 0.70956362 0.63736799]]
(3, 6)
(6,)
(3,)


In [13]:
constraints = [0 <= x, A_param @ x == b]

def obj(a,x):
    out = 0
    idx = np.arange(x.shape[0]) 
    for i in np.arange(a.shape[0]): # loop over each item i
        # extract the indices of vector x that correspond to the item i
        # size of x_idcs must be m
        x_idcs = idx[idx % m == i]
        for j in np.arange(a.shape[1]): #loop over each element of a^{i}
            # a_row corresponds to the index ji and a_col corresponds to the index ki
            a_row, a_col = (math.floor(j / n), j % n)
            out += a[i][j] * cp.minimum(x[x_idcs[a_row]], x[x_idcs[a_col]])
    return out

objective = cp.Maximize(obj(a,x))

prob = cp.Problem(objective, constraints)

result = prob.solve()

            
x = x.value.reshape(n,m)

print(x)

[[0. 1.]
 [0. 1.]
 [0. 1.]]
